# Semantic Text Similarity
Este modelo utiliza gensim para convertir pares de vectores + puntuaciones en vectores (word embeddings).
Dado un dataset, infiere la puntuación de similitud entre ambas frases.

In [1]:
# Requisitos
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np

In [2]:
# Tipado
from typing import Tuple, List

In [3]:
# Cargar stopwords en Catalan
# STOPWORDS_CA = {"a", "abans", "ací", "ah", "així", "això", "al", "aleshores", "algun", "alguna", "algunes", "alguns", "alhora", "allà", "allí", "allò", "als", "altra", "altre", "altres", "amb", "ambdues", "ambdós", "anar", "ans", "apa", "aquell", "aquella", "aquelles", "aquells", "aquest", "aquesta", "aquestes", "aquests", "aquí", "baix", "bastant", "bé", "cada", "cadascuna", "cadascunes", "cadascuns", "cadascú", "com", "consegueixo", "conseguim", "conseguir", "consigueix", "consigueixen", "consigueixes", "contra", "d'un", "d'una", "d'unes", "d'uns", "dalt", "de", "del", "dels", "des", "des de", "després", "dins", "dintre", "donat", "doncs", "durant", "e", "eh", "el", "elles", "ells", "els", "em", "en", "encara", "ens", "entre", "era", "erem", "eren", "eres", "es", "esta", "estan", "estat", "estava", "estaven", "estem", "esteu", "estic", "està", "estàvem", "estàveu", "et", "etc", "ets", "fa", "faig", "fan", "fas", "fem", "fer", "feu", "fi", "fins", "fora", "gairebé", "ha", "han", "has", "haver", "havia", "he", "hem", "heu", "hi", "ho", "i", "igual", "iguals", "inclòs", "ja", "jo", "l'hi", "la", "les", "li", "li'n", "llarg", "llavors", "m'he", "ma", "mal", "malgrat", "mateix", "mateixa", "mateixes", "mateixos", "me", "mentre", "meu", "meus", "meva", "meves", "mode", "molt", "molta", "moltes", "molts", "mon", "mons", "més", "n'he", "n'hi", "ne", "ni", "no", "nogensmenys", "només", "nosaltres", "nostra", "nostre", "nostres", "o", "oh", "oi", "on", "pas", "pel", "pels", "per", "per que", "perquè", "però", "poc", "poca", "pocs", "podem", "poden", "poder", "podeu", "poques", "potser", "primer", "propi", "puc", "qual", "quals", "quan", "quant", "que", "quelcom", "qui", "quin", "quina", "quines", "quins", "què", "s'ha", "s'han", "sa", "sabem", "saben", "saber", "sabeu", "sap", "saps", "semblant", "semblants", "sense", "ser", "ses", "seu", "seus", "seva", "seves", "si", "sobre", "sobretot", "soc", "solament", "sols", "som", "son", "sons", "sota", "sou", "sóc", "són", "t'ha", "t'han", "t'he", "ta", "tal", "també", "tampoc", "tan", "tant", "tanta", "tantes", "te", "tene", "tenim", "tenir", "teniu", "teu", "teus", "teva", "teves", "tinc", "ton", "tons", "tot", "tota", "totes", "tots", "un", "una", "unes", "uns", "us", "va", "vaig", "vam", "van", "vas", "veu", "vosaltres", "vostra", "vostre", "vostres", "érem", "éreu", "és", "éssent", "últim", "ús"}
STOPWORDS_CA = {"a", "al", "el", "la", "els", "les", "de", "un", "una", "algun", "alguna", }

In [4]:
# Definir función de pre-procesado
def preprocess(sentence: str) -> List[str]:
    preprocessed = simple_preprocess(sentence) # Tokenización y normalización, lematización, minúsculas
    # Eliminar stopwords
    preprocessed = [token for token in preprocessed if token not in STOPWORDS_CA]
    return preprocessed

In [5]:
"""
# Modelos pre-entrenados
# WV_MODEL_PATH = "/Users/salva/Downloads/cc.ca.300.bin.gz"
WV_MODEL_PATH = '/Users/salva/Downloads/cc.ca.300.vec.gz'
import gensim
wv_model =  gensim.models.KeyedVectors.load_word2vec_format(WV_MODEL_PATH, binary=False)
wv_model
"""

'\n# Modelos pre-entrenados\n# WV_MODEL_PATH = "/Users/salva/Downloads/cc.ca.300.bin.gz"\nWV_MODEL_PATH = \'/Users/salva/Downloads/cc.ca.300.vec.gz\'\nimport gensim\nwv_model =  gensim.models.KeyedVectors.load_word2vec_format(WV_MODEL_PATH, binary=False)\nwv_model\n'

In [6]:
from gensim.models.fasttext import FastTextKeyedVectors
#cargar como map:
wv_model = FastTextKeyedVectors.load('/home/taya/Desktop/cc.ca.gensim.bin', mmap='r')

In [7]:
"""
# Ejemplo de 10 pares de oraciones con puntuación de similitud asociada
#No se usa nunca en el codigo
input_pairs = [
    ('M\'agrada el futbol', 'Disfruto veient partits de futbol', 4),
    ('El cel està despejat', 'Fa un dia bonic', 4.5),
    ('M\'encanta viatjar', 'Explorar nous llocs és una passió', 3.5),
    ('Prefereixo l\'estiu', 'No m\'agrada el fred de l\'hivern', 2.5),
    ('Tinc gana', 'Què hi ha per sopar?', 2),
    ('La música em relaxa', 'Escoltar música és una teràpia', 3),
    ('El llibre és emocionant', 'No puc deixar de llegir-lo', 4),
    ('M\'agrada la pizza', 'És el meu menjar preferit', 4.5),
    ('Estic cansat', 'Necessito fer una migdiada', 1.5),
    ('Avui fa molta calor', 'És un dia sofocant', 3.5)
    ]

"""

"\n# Ejemplo de 10 pares de oraciones con puntuación de similitud asociada\n#No se usa nunca en el codigo\ninput_pairs = [\n    ('M'agrada el futbol', 'Disfruto veient partits de futbol', 4),\n    ('El cel està despejat', 'Fa un dia bonic', 4.5),\n    ('M'encanta viatjar', 'Explorar nous llocs és una passió', 3.5),\n    ('Prefereixo l'estiu', 'No m'agrada el fred de l'hivern', 2.5),\n    ('Tinc gana', 'Què hi ha per sopar?', 2),\n    ('La música em relaxa', 'Escoltar música és una teràpia', 3),\n    ('El llibre és emocionant', 'No puc deixar de llegir-lo', 4),\n    ('M'agrada la pizza', 'És el meu menjar preferit', 4.5),\n    ('Estic cansat', 'Necessito fer una migdiada', 1.5),\n    ('Avui fa molta calor', 'És un dia sofocant', 3.5)\n    ]\n\n"

In [8]:
from datasets import load_dataset
# Text Similarity (STS) dataset (principal per la Pràctica 4)
train = load_dataset("projecte-aina/sts-ca", split="train")
test = load_dataset("projecte-aina/sts-ca", split="test")
val = load_dataset("projecte-aina/sts-ca", split="validation")
all_data = load_dataset("projecte-aina/sts-ca", split="all")
all_data

Dataset({
    features: ['id', 'sentence_1', 'sentence_2', 'label'],
    num_rows: 3073
})

Preprocesamiento y construccion del diccionario:

In [9]:
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [simple_preprocess(d["sentence_1"]) for d in all_data] #lista de listas que son oraciones lematizadas
sentences_2_preproc = [simple_preprocess(d["sentence_2"]) for d in all_data]
scores = [d["label"] for d in all_data]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc, scores))#lista de tuplas que son ([palabras or1], [pal or 2], score)
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc #todas las oraciones juntas
diccionario = Dictionary(sentences_pairs_flattened) # diccionario donde cada palabra tiene un indice unico
diccionario

In [10]:
print(sentence_pairs[0])

(['atorga', 'per', 'primer', 'cop', 'les', 'mencions', 'encarna', 'sanahuja', 'la', 'inclusió', 'de', 'la', 'perspectiva', 'de', 'gènere', 'en', 'docència', 'universitària'], ['creen', 'la', 'menció', 'encarna', 'sanahuja', 'la', 'inclusió', 'de', 'la', 'perspectiva', 'de', 'gènere', 'en', 'docència', 'universitària'], 3.5)


construccion de la metriz TF-IDF:

In [11]:
# Cálculo de los pesos TF-IDF para las oraciones pre-procesadas
corpus = [diccionario.doc2bow(sent) for sent in sentences_pairs_flattened]
"""
Por ejemplo, si sent es ['hola', 'mundo', 'hola'], el resultado de diccionario.doc2bow(sent) podría ser [(0, 2), (1, 1)], donde 0 es el índice de "hola" y 1 es el índice de "mundo", indicando que "hola" aparece 2 veces y "mundo" aparece 1 vez.
corpus = El resultado es una lista de representaciones de bolsa de palabras, donde cada elemento corresponde a una oración en el conjunto de datos.
"""
modelo_tfidf = TfidfModel(corpus) #transformar el corpus en una representación que refleja la importancia de las palabras en cada documento en relación con el corpus completo.

Modelos de dimensión reducida:

In [12]:
wv_model_50d = {
    word: wv_model[word][:50]
    for word in wv_model.index_to_key
}

wv_model_100d = {
    word: wv_model[word][:100]
    for word in wv_model.index_to_key
}
wv_model_150d = {
    word: wv_model[word][:150]
    for word in wv_model.index_to_key
}

Aregación:

In [13]:
def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel, model = wv_model) -> Tuple[List[np.ndarray], List[float]]:
    """
    lo que hace es que coge una oracion preprocesada, para cada palabra saca sus pesos TF-IDF y su vector en el embeding
    """
    bow = dictionary.doc2bow(sentence_preproc)#cuenta la frecuencia de cada palabra en la oracion
    tf_idf = tf_idf_model[bow] 
    vectors, weights = [], []
    for word_index, weight in tf_idf:
        word = dictionary.get(word_index)
        if word in model:
            vectors.append(model[word])
            weights.append(weight)
    return vectors, weights

def map_pairs(wv_model2, sentence_pairs: List[Tuple[str, str, float]],dictionary: Dictionary = None, tf_idf_model: TfidfModel = None,) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    """
    Mapea los tripletes de oraciones a listas de (x, y), (pares de vectores, score)
    :param sentence_pairs: lista de tuplas que son ([palabras or1], [palabras or2], score)
    :param dictionary: diccionario donde cada palabra tiene un indice unico
    :param tf_idf_model: objeto TfidfModel que da los pesos de las palabras (se puede indexar con un bag of words)
    :return: lista de ((vector1, vector2), similitud), donde vector1 y vector2 cambian en funcion de:
        si tf_idf_model is not None:
                para cada elemento de sentence_pairs devuelve el vector embeding promediado de manera ponderada por los pesos de la matriz TF-IDF de las palabras de las oraciones 1 y 2.
        si tf_idf_model is not None
            el promedio de los vectores de embeding de las palabras que componen cada una de las oraciones
    """
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess(sentence_1) if isinstance(sentence_1, str) else sentence_1 # se procesa el texto antes de aplicar map_pairs entonces sentence_1 es una lista de tokens y ya nose vuelve a preprocesar
        sentence_2_preproc = preprocess(sentence_2) if isinstance(sentence_2, str) else sentence_2
        # Si usamos TF-IDF
        if tf_idf_model is not None:
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model,model =  wv_model2, )
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, model = wv_model2 )
            vector1 = np.average(vectors1, weights=weights1, axis=0, ) #Esta función calcula el promedio de un conjunto de valores. Si se proporciona un argumento weights, el promedio se calcula de manera ponderada, lo que significa que cada valor contribuye al promedio de acuerdo con su peso correspondiente.
            vector2 = np.average(vectors2, weights=weights2, axis=0, )
        else:
            # Cálculo del promedio de los word embeddings
            vectors1 = [wv_model2[word] for word in sentence_1_preproc if word in wv_model2]
            vectors2 = [wv_model2[word] for word in sentence_2_preproc if word in wv_model2]
            vector1 = np.mean(vectors1, axis=0)
            vector2 = np.mean(vectors2, axis=0)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [14]:
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped_no_tfidf = map_pairs(wv_model, sentence_pairs, tf_idf_model=None, dictionary=diccionario, )
mapped = map_pairs(wv_model,sentence_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped[0]

((array([-7.22131877e-03, -4.88683421e-03,  2.71017708e-02,  2.32332627e-02,
         -9.60097482e-03, -3.16095435e-03,  2.90225599e-02, -1.75413820e-02,
          2.93095319e-02, -1.60574403e-02, -6.04936805e-03,  1.49908545e-02,
          1.00934507e-02,  1.84449753e-02,  2.16156266e-02,  2.13238810e-02,
          8.69774586e-03,  6.13958934e-02,  2.18719114e-02,  1.01426407e-02,
          1.16837708e-02,  3.24588305e-03, -1.32856906e-02,  5.77104877e-02,
          1.54627187e-02,  2.13443526e-02, -3.82471218e-02,  6.23983637e-03,
          7.31161386e-04,  8.99225741e-03, -4.87626204e-03,  1.08773269e-02,
          1.30313145e-02, -3.86450925e-03,  7.23370200e-03, -1.75266524e-02,
         -9.09778208e-03,  4.43412138e-02, -4.31998409e-04,  6.25044253e-04,
         -1.13920750e-02, -1.82465011e-02, -8.11444328e-03, -8.18518457e-03,
         -3.54176235e-03, -1.60820262e-01,  7.74505797e-03,  9.80261699e-03,
          8.53058034e-03, -1.23878019e-02,  1.24134202e-02, -2.39070017e-03,

De dimensión reducida: <sb>

sí usando pesos TF-IDF

In [15]:
mapped_50 = map_pairs(wv_model_50d,sentence_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped_100 = map_pairs(wv_model_100d,sentence_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped_150 = map_pairs(wv_model_150d,sentence_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, )

In [16]:
print(mapped_no_tfidf[0][0][0].shape)
print(mapped[0][0][0].shape)
print(mapped_50[0][0][0].shape)
print(mapped_100[0][0][0].shape)
print(mapped_150[0][0][0].shape)

(300,)
(300,)
(50,)
(100,)
(150,)


Diferentes modelos:

In [45]:
# Definir el Modelo
import tensorflow as tf

def build_and_compile_model(hidden_size: int = 128, embedding_size: int = 300, learning_rate: float = 0.001) -> tf.keras.Model:
    """
    Esto crea una red neuronal de manera que al entrenarla las distancias coseno cuadren con la etiqueta real
    hidden_size: Tamaño de capas ocultas (no se usa en este código)
    embedding_size: Dimensión de los vectores de entrada (300)
    learning_rate: Tasa de aprendizaje para el optimizador
    """
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,)) #los pares de vectores a comparar
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Capa oculta, con funcion de activacion lineal, tiene como objetivo proyectar los vectores de entrada en un nuevo espacio.
    """
    La capa oculta (en este caso, la capa densa) tiene pesos que se ajustan durante el entrenamiento.
    Estos pesos son los que transforman los vectores de entrada en los vectores proyectados
    """
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        # activation='tanh',
        kernel_initializer=tf.keras.initializers.Identity(),# inicializa los pesos de la capa como una matriz identidad
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    #aplica la capa de proyeccion a los dos vectores de entrada
    projected_1 = first_projection(input_1)
    projected_2 = first_projection(input_2)
    """
    # Compute the cosine distance
    projected_1 = tf.linalg.l2_normalize(projected_1, axis=1, ) #Normaliza ambos vectores para que tengan magnitud 1, necesario para el cálculo de similitud coseno
    projected_2 = tf.linalg.l2_normalize(projected_2, axis=1, )
    output = 2.5 * (1.0 + tf.reduce_sum(projected_1 * projected_2, axis=1, ))
    """ 
    #lo comentado es del profe y no va. Esto es del Chat############################################################################################
    normalize = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))
    projected_1 = normalize(projected_1)
    projected_2 = normalize(projected_2)
    output = tf.keras.layers.Lambda(lambda tensors: 2.5 * (1.0 + tf.reduce_sum(tensors[0] * tensors[1], axis=1)))([projected_1, projected_2])
    ############################################################################################################################################
    # Definir el modelo con las capas de entrada y salida
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output) #Durante el entrenamiento, Keras ajusta los pesos de la capa oculta para minimizar la función de pérdida definida (en este caso, el error absoluto medio).

    # Compilar el modelo
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate))

    return model

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)

En las transparencias para el modelo 1 pone este codigo:

In [18]:
def build_model_aggregated(embedding_dim: int, hidden_size: int = 128, dropout_rate: float = 0.3) -> tf.keras.Model:
    input_1 = tf.keras.Input(shape=(embedding_dim,), name="input_vector_1")
    input_2 = tf.keras.Input(shape=(embedding_dim,), name="input_vector_2")
    concatenated = tf.keras.layers.Concatenate(axis=-1)([input_1, input_2])
    x = tf.keras.layers.BatchNormalization()(concatenated)
    x = tf.keras.layers.Dense(hidden_size, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    output = tf.keras.layers.Dense(1)(x) # Activació lineal per a regressió
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

#model_agg.fit([X1_train, X2_train], Y_train, epochs=..., batch_size=...)

🧠 Modelo build_model_aggregated: Concatenación + Red Neuronal Densa
🏗️ Arquitectura:

    Toma dos vectores de entrada (input_vector_1, input_vector_2).

    Los concatena (Concatenate), por lo que la dimensión del vector combinado es el doble del embedding_dim.

    Aplica:

        BatchNormalization

        Dense con ReLU

        Otro BatchNormalization

        Dropout

        Una capa de salida densa sin activación (regresión lineal).

    Se entrena con MSE (Mean Squared Error).

🧾 Objetivo implícito:

    Aprender una función no lineal entre los vectores concatenados y la puntuación objetivo (p. ej., similitud STS, afinidad, etc.).

    Aprende una transformación compleja basada en composición conjunta de los dos vectores.

📌 Ventajas:

    Flexibilidad para aprender patrones complejos.

    Permite capturar interacciones no lineales entre los dos vectores.

❗ Consideraciones:

    Puede sobreajustarse si el dataset es pequeño.

    Requiere más parámetros, por lo tanto más datos para entrenar bien.

🧠 build_and_compile_model: Proyección + Similitud Coseno
🏗️ Arquitectura:

    Aplica una capa densa compartida (misma proyección) a cada vector por separado. Inicialmente es una matriz identidad.

    Los normaliza a magnitud 1.

    Calcula la similitud coseno como cos(θ) = dot(product).

    La salida es 2.5 * (1 + similitud_coseno), lo cual transforma el rango [-1, 1] a [0, 5].

    Se entrena con MAE (Mean Absolute Error).

🧾 Objetivo implícito:

    Aprender una proyección donde la similitud coseno refleje la puntuación deseada (por ejemplo, cuán similares son dos textos o frases).

    Optimiza directamente sobre una función interpretable (similitud coseno), útil para tareas tipo semantic textual similarity (STS).

📌 Ventajas:

    Muy interpretativo.

    Más simple y menos propenso a sobreajuste.

    Funciona bien cuando la relación entre embeddings y similitud es principalmente angular (coseno).

❗ Consideraciones:

    Menor capacidad expresiva que el Modelo 1.

    Asume que la similitud se puede modelar bien con una proyección lineal + coseno.

In [19]:

def build_and_compile_model2(embedding_size: int = 300, learning_rate: float = 0.001) -> tf.keras.Model:
    # Input layer
    input_1 = tf.keras.Input(shape=(embedding_size,), name="input_vector_1")
    input_2 = tf.keras.Input(shape=(embedding_size,), name="input_vector_2")

    # hidden layer
    first_projection_layer = tf.keras.layers.Dense(
        embedding_size,
        activation='tanh',
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
        name="projection_layer"
    )
    dropout = tf.keras.layers.Dropout(0.3, name="projection_dropout")
    projected_1_dense = dropout(first_projection_layer(input_1))
    projected_2_dense = dropout(first_projection_layer(input_2))

    # Normalize the projected vectors using Lambda layers
    normalized_1 = tf.keras.layers.Lambda(
        lambda x: tf.linalg.l2_normalize(x, axis=1), name="normalize_1"
    )(projected_1_dense)
    normalized_2 = tf.keras.layers.Lambda(
        lambda x: tf.linalg.l2_normalize(x, axis=1), name="normalize_2"
    )(projected_2_dense)

    # Compute the custom similarity score using a Lambda layer
    similarity_sum = tf.keras.layers.Lambda(
        lambda x: tf.reduce_sum(x[0] * x[1], axis=1, keepdims=True), name="similarity_sum"
    )([normalized_1, normalized_2])

    output = tf.keras.layers.Lambda(
        lambda x: 0.5 * (1.0 + x), name="output_scaling" #cambiar 0.5 por 2.5 para que este entre 0 y 5 
    )(similarity_sum)

    # Definir el modelo con las capas de entrada y salida
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output, name="similarity_model")

    # Compilar el modelo
    model.compile(
        loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    )

    return model


| Característica             | Modelo 1: Cosine (`build_and_compile_model2`) | Modelo 2: MLP (`build_model_aggregated`) |
| -------------------------- | -------------------------------------------- | ---------------------------------------- |
| Tipo de entrada            | 2 vectores                                   | 2 vectores concatenados                  |
| Proyección                 | Capa densa compartida                        | Dense normal (no compartida)             |
| Normalización L2           | ✅ sí                                         | ❌ no                                     |
| Métrica implícita          | Cosine similarity                            | No definida; aprende desde los datos     |
| Salida                     | Escalado de coseno (rango 0-1 o 0-5)         | Escalar libre (regresión lineal)         |
| Capacidad expresiva        | Limitada (coseno + proyección)               | Alta (MLP)                               |
| Interpretabilidad          | Alta                                         | Media                                    |
| Velocidad de entrenamiento | Más rápido                                   | Más lento                                |


.

| Aspecto                         | build_and_compile_model                  | `build_and_compile_model2`                              |
| ------------------------------- | ----------------------------------------- | ------------------------------------------------------ |
| **Activación en la proyección** | Ninguna (lineal)                          | `tanh` (no lineal)                                     |
| **Regularización**              | No hay                                    | Sí, con `Dropout`                                      |
| **Normalización**               | Directamente con `tf.linalg.l2_normalize` | Igual, pero encapsulada en `Lambda` layers con nombres |
| **Similitud coseno**            | `2.5 * (1.0 + coseno)`                    | `0.5 * (1.0 + coseno)`                                 |
| **Escalado de salida**          | Rango `[0, 5]`                            | Rango `[0, 1]`                                         |
| **Perdida**                     | `mean_absolute_error`                     | `mean_squared_error`                                   |
| **Estilo**                      | Más directo, menos modular                | Más claro, modular, con nombres de capas               |


.

Training y evaluación:

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr

def evaluate_model(model, X1_test, X2_test, Y_test, name=""):
    y_pred = model.predict([X1_test, X2_test]).squeeze()
    y_true = Y_test.squeeze()

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    pearson, _ = pearsonr(y_true, y_pred)
    spearman, _ = spearmanr(y_true, y_pred)

    print(f"\n🔎 Resultados para el modelo '{name}':")
    print(f"MSE:      {mse:.4f}")
    print(f"RMSE:     {rmse:.4f}")
    print(f"MAE:      {mae:.4f}")
    print(f"Pearson:  {pearson:.4f}")
    print(f"Spearman: {spearman:.4f}")

    return {
        "name": name,
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "pearson": pearson,
        "spearman": spearman
    }


In [21]:
# Definir constantes de entrenamiento
batch_size: int = 64
num_epochs: int = 64
train_val_split: float = 0.8

In [22]:
# Obtener x_train e y_train
train_slice: int = int(len(mapped) * train_val_split)

In [23]:
train_slice

2458

In [24]:
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    """
    Otiene las matrices X_1 (N x d) , X_2 (N x d), e Y (n) a partir de listas de parejas de vectores de oraciones - Listas de (d, d, 1)
    :param pair_list: lista que devuelve map_pairs(), lista de ((vector1, vector2), similitud), sonde vector1 y 2 son vectores agregados
    :return:
    transforma una lista de pares de vectores y puntuaciones (como los que se usan en tareas de similaridad semántica tipo STS) en el formato adecuado para alimentar a un modelo de aprendizaje automático.
    """
    _x, _y = zip(*pair_list) #_x: lista de tuplas (embedding_1, embedding_2), _y: lista de etiquetas
    _x_1, _x_2 = zip(*_x)#_x_1: todos los embedding_1,  _x_2: todos los embedding_2
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, ) / 5.0

In [25]:
# Obtener las listas de train y test USANDO TF-IDF
x_train, y_train = pair_list_to_x_y(mapped[:train_slice])
x_val, y_val = pair_list_to_x_y(mapped[train_slice:])

In [26]:
# Obtener las listas de train y test SIN USAR TF-IDF
x_train_normal, y_train_normal = pair_list_to_x_y(mapped[:train_slice])
x_val_normal, y_val_normal = pair_list_to_x_y(mapped[train_slice:])

In [27]:
x_train_50, y_train_50 = pair_list_to_x_y(mapped_50[:train_slice])
x_val_50, y_val_50 = pair_list_to_x_y(mapped_50[train_slice:])

x_train_100, y_train_100 = pair_list_to_x_y(mapped_100[:train_slice])
x_val_100, y_val_100 = pair_list_to_x_y(mapped_100[train_slice:])

x_train_150, y_train_150 = pair_list_to_x_y(mapped_150[:train_slice])
x_val_150, y_val_150 = pair_list_to_x_y(mapped_150[train_slice:])

In [28]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

2025-05-25 17:07:34.108625: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [29]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset_normal = tf.data.Dataset.from_tensor_slices((x_train_normal, y_train_normal))
train_dataset_normal = train_dataset_normal.shuffle(buffer_size=len(x_train_normal)).batch(batch_size)

val_dataset_normal = tf.data.Dataset.from_tensor_slices((x_val_normal, y_val_normal))
val_dataset_normal = val_dataset_normal.batch(batch_size)

### Evaluacion

#### Distancia COS:

In [30]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
from scipy.stats import pearsonr
from scipy import spatial

In [31]:
val_normal = mapped_no_tfidf[train_slice:]
val = mapped[train_slice:]
val_50 = mapped_50[train_slice:]
val_100 = mapped_100[train_slice:]
val_150 = mapped_150[train_slice:]

In [32]:
y_pred_baseline = []
y_pred_normal = []
y_pred_50 = []
y_pred_100 = []
y_pred_150 = []
for j in range(len(val)):
    i = val[j]
    v1, v2 = i[0] 
    d = 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)

    k = val_normal[j]
    v1, v2 = k[0] 
    d = 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_normal.append(d)

    m = val_50[j]
    v1, v2 = m[0] 
    d = 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_50.append(d)

    l = val_100[j]
    v1, v2 = l[0] 
    d = 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_100.append(d)

    e = val_150[j]
    v1, v2 = e[0] 
    d = 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_150.append(d)

# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(np.array(y_pred_baseline), y_val.flatten())
correlation_normal, _ = pearsonr(np.array(y_pred_normal), y_val_normal.flatten())
correlation_50, _ = pearsonr(np.array(y_pred_50), y_val_50.flatten())
correlation_100, _ = pearsonr(np.array(y_pred_100), y_val_100.flatten())
correlation_150, _ = pearsonr(np.array(y_pred_150), y_val_150.flatten())

# Imprimir el coeficiente de correlación de Pearson
print(f"Usando media clásica la correlación de Pearson es: {correlation}")
print(f"Usando media ponderada con TF-IDF la correlación de Pearson es: {correlation_normal}")
print(f"Usando media ponderada con TF-IDF y dimensión 50, la correlación de Pearson es: {correlation_50}")
print(f"Usando media ponderada con TF-IDF y dimensión 100, la correlación de Pearson es: {correlation_100}")
print(f"Usando media ponderada con TF-IDF y dimensión 150, la correlación de Pearson es: {correlation_150}")


Usando media clásica la correlación de Pearson es: 0.43930761675943547
Usando media ponderada con TF-IDF la correlación de Pearson es: 0.3689169387565914
Usando media ponderada con TF-IDF y dimensión 50, la correlación de Pearson es: 0.41854519762908476
Usando media ponderada con TF-IDF y dimensión 100, la correlación de Pearson es: 0.4308664019477864
Usando media ponderada con TF-IDF y dimensión 150, la correlación de Pearson es: 0.4422013561274339


In [33]:
"""#la del profe: (la de arriba es la misma pero la he modificado para ver las dos correlaciones a la vez)
val = mapped[train_slice:]
y_pred_baseline = []
y_pred_normal = []
for (v1, v2), dist in val:
    d = 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(np.array(y_pred_baseline), y_val.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")
"""

'#la del profe: (la de arriba es la misma pero la he modificado para ver las dos correlaciones a la vez)\nval = mapped[train_slice:]\ny_pred_baseline = []\ny_pred_normal = []\nfor (v1, v2), dist in val:\n    d = 1.0 - spatial.distance.cosine(v1, v2)\n    y_pred_baseline.append(d)\n# Calcular la correlación de Pearson entre las predicciones y los datos de prueba\ncorrelation, _ = pearsonr(np.array(y_pred_baseline), y_val.flatten())\n# Imprimir el coeficiente de correlación de Pearson\nprint(f"Correlación de Pearson: {correlation}")\n'

#### Prueba del embeding aprendido:

In [ ]:
model_no_cos = build_model_aggregated(embedding_dim=300)
model_no_cos.fit([x_train], y_train, epochs=num_epochs, batch_size=batch_size)

Epoch 1/64


/home/taya/Desktop/myenv/lib/python3.12/site-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_vector_1', 'input_vector_2']
Received: inputs=(('Tensor(shape=(None, 300))', 'Tensor(shape=(None, 300))'),)
  warnings.warn(msg)


39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 2.3664 - mae: 1.1879 - root_mean_squared_error: 1.5340
Epoch 2/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9782 - mae: 0.7739 - root_mean_squared_error: 0.9889
Epoch 3/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7932 - mae: 0.6947 - root_mean_squared_error: 0.8904
Epoch 4/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5493 - mae: 0.5778 - root_mean_squared_error: 0.7409
Epoch 5/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4561 - mae: 0.5207 - root_mean_squared_error: 0.6745
Epoch 6/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3417 - mae: 0.4474 - root_mean_squared_error: 0.5845
Epoch 7/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.3004 - mae: 0.4108 - root_mean_squared_error: 0.5479
Epoch 8/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.2426 - mae: 0.3695 - root_mean_squared_error: 0.4917
Epoch 9/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.1774 - mae: 0.3216 - root_mean_squared_error

In [35]:
X1, X2 = x_val
evaluate_model(model_no_cos, X1, X2, y_val, name="Linealmodel no cos()")

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step

🔎 Resultados para el modelo 'Linealmodel no cos()':
MSE:      0.0279
RMSE:     0.1672
MAE:      0.1280
Pearson:  0.3419
Spearman: 0.3337


{'name': 'Linealmodel no cos()',
 'mse': 0.027944477,
 'rmse': 0.16716602,
 'mae': 0.12803978,
 'pearson': 0.3418520310011242,
 'spearman': 0.33370564078195586}

In [ ]:
#tf.keras.utils.plot_model(model_no_cos, show_shapes=True, show_layer_activations=True, )

In [ ]:
#model_no_cos_50 = build_model_aggregated(embedding_dim=50)
#model_no_cos.fit([x_train_50], y_train_50, epochs=num_epochs, batch_size=batch_size)

Epoch 1/64


/home/taya/Desktop/myenv/lib/python3.12/site-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_vector_1', 'input_vector_2']
Received: inputs=(('Tensor(shape=(None, None))', 'Tensor(shape=(None, None))'),)
  warnings.warn(msg)
2025-05-25 17:33:02.090854: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [64,100] vs. [600]
	 [[{{function_node __inference_one_step_on_data_55655}}{{node gradient_tape/functional_1/batch_normalization_1/batchnorm/mul_1/BroadcastGradientArgs}}]]


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/functional_1/batch_normalization_1/batchnorm/mul_1/BroadcastGradientArgs defined at (most recent call last):
  File "/usr/lib/python3.12/runpy.py", line 198, in _run_module_as_main

  File "/usr/lib/python3.12/runpy.py", line 88, in _run_code

  File "/home/taya/Desktop/myenv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/taya/Desktop/myenv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/taya/Desktop/myenv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/taya/Desktop/myenv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  

In [47]:
model_lin = build_and_compile_model()
model_lin.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.9752 - val_loss: 3.7108
Epoch 2/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.6279 - val_loss: 3.6226
Epoch 3/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.5095 - val_loss: 3.5766
Epoch 4/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.4266 - val_loss: 3.5462
Epoch 5/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.3611 - val_loss: 3.5241
Epoch 6/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.3055 - val_loss: 3.5071
Epoch 7/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.2580 - val_loss: 3.4937
Epoch 8/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.2178 - val_loss: 3.4825
Epoch 9/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.1824 - val_loss: 3.4730
Epoch 10/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.1502 - val_loss: 3.4649
Epoch 11/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.1230 - val_loss: 3.4580
Epoch 12/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.0975 - v

In [49]:
evaluate_model(model_lin, X1, X2, y_val)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

🔎 Resultados para el modelo '':
MSE:      12.2433
RMSE:     3.4990
MAE:      3.3332
Pearson:  0.3417
Spearman: 0.3970


{'name': '',
 'mse': 12.243323,
 'rmse': 3.499046,
 'mae': 3.3331766,
 'pearson': 0.341672256901909,
 'spearman': 0.397042571697594}

In [ ]:
"""
y_pred: tf.RaggedTensor = model_lin.predict(x_val)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(y_pred.flatten(), y_val.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")
"""

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Correlación de Pearson: 0.341672256901909


In [ ]:
# Construir y compilar el modelo
model_no_lin = build_and_compile_model2()
# tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True, )
#print(model.summary())
# Entrenar el modelo
model_no_lin.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

In [41]:
#from scipy.stats import pearsonr
# Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
y_pred: tf.RaggedTensor = model_no_lin.predict(x_val)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(y_pred.flatten(), y_val.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Correlación de Pearson: 0.5301249838419955


lo mismo pero para el train:

In [44]:
"""
#from scipy.stats import pearsonr
# Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
y_pred: tf.RaggedTensor = model_no_lin.predict(x_train)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(y_pred.flatten(), y_train.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")
"""

'\n#from scipy.stats import pearsonr\n# Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.\ny_pred: tf.RaggedTensor = model_no_lin.predict(x_train)\n# Calcular la correlación de Pearson entre las predicciones y los datos de prueba\ncorrelation, _ = pearsonr(y_pred.flatten(), y_train.flatten())\n# Imprimir el coeficiente de correlación de Pearson\nprint(f"Correlación de Pearson: {correlation}")\n'